### Get data from Steam Community Market API and save to csv

In [5]:
import re
import requests
import pandas as pd

dailyCookie = "76561199704981720%7C%7CeyAidHlwIjogIkpXVCIsICJhbGciOiAiRWREU0EiIH0.eyAiaXNzIjogInI6MTcwQl8yNDkzRTBDMF80QkU4NSIsICJzdWIiOiAiNzY1NjExOTk3MDQ5ODE3MjAiLCAiYXVkIjogWyAid2ViOmNvbW11bml0eSIgXSwgImV4cCI6IDE3MTg1NjE2MjksICJuYmYiOiAxNzA5ODM0MjI5LCAiaWF0IjogMTcxODQ3NDIyOSwgImp0aSI6ICIxNEY5XzI0OTNFMEMxX0FEMkFFIiwgIm9hdCI6IDE3MTgzNjI3ODYsICJydF9leHAiOiAxNzM2NjE4ODA2LCAicGVyIjogMCwgImlwX3N1YmplY3QiOiAiODEuMTA1LjIwMS41NyIsICJpcF9jb25maXJtZXIiOiAiOTAuMTk3Ljc5LjEzMyIgfQ.e_-hJhZLVIE2Vh24baY6-qLNMhcLl_oKNyj0UQsLWVXeTA56Sy0cBh9WRHRdUEQ3JuV9p3-YVUHcloskwOdaDw"
items = ["Glove Case Key", "Officer Jacques Beltram | Gendarmerie Nationale"]

def fetch_item_to_df(item):
    # get historical price data of item from API
    url = "https://steamcommunity.com/market/pricehistory/"
    params = {
        'country': 'US',
        'currency': '1',
        'appid': '730',
        'market_hash_name': item
    }
    cookies = {'steamLoginSecure': dailyCookie}
    
    response = requests.get(url, params=params, cookies=cookies)
    jsonData = response.json()

    # convert and clean data to dataframe object
    historical_prices = jsonData['prices']
    historical_prices_cleaned = pd.DataFrame(historical_prices, columns=['date', 'price_pounds', 'volume'])
    historical_prices_cleaned['date'] = pd.to_datetime(historical_prices_cleaned['date'].str[:-7], format='%b %d %Y')
    return historical_prices_cleaned

def sanitize_filename(filename):
    """Sanitizes the filename to ensure it is valid for most operating systems."""
    filename = re.sub(r'[\\/*?:"<>|]', '_', filename)  # Replace disallowed characters with underscore
    filename = re.sub(r'\s+', '_', filename)  # Replace spaces with underscores
    return filename

def save_item_to_csv(item):
    csvData = './data/'+ sanitize_filename(item) +'.csv'
    fetch_item_to_df(item).to_csv(csvData, index=False)


# fetch and save data in items to csv
for index, item in enumerate(items):
    save_item_to_csv(item)

